# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

experiment_name = 'mnist-automl-experiment'
experiment = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Attach an AmlCompute cluster

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Udacity subscription does not allow to create GPU instance
cpu_cluster = "mnist-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS2_V2",
                                                           vm_priority='lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster, compute_config)

compute_target.wait_for_completion(show_output=True)

## Dataset

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. The digits have been size-normalized and centered in a fixed-size image. The images were centered in a 28x28 image by computing the center of mass of the pixels. The data is stored in a very simple file format designed for storing vectors and multidimensional matrices.

Reference: http://yann.lecun.com/exdb/mnist/

Run the `hyperparameter_tuning.ipynb` Dataset section to register the data before obtain it.

In [ ]:
from azureml.core import Dataset

# Get the already existing dataset
train_ds = Dataset.get_by_name(ws, "mnist-train")
test_ds = Dataset.get_by_name(ws, "mnist-test")

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=61,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='labels',
    n_cross_validations=5,
    max_concurrent_iterations=4
    )

In [ ]:
# Submit the automl run to the experiment
automl_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve the best automl model
best_run, best_model = automl_run.get_output()
best_run_metrics = best_run.get_metrics()

print(f"Best run: {best_run}\n")
print(f"Best model: {best_model}\n")
print(f"Best run metrics: {best_run_metrics}\n")
print(f"Associated files: {best_run.get_file_names()}\n")

In [ ]:
# TODO: Download the model
# Downloading model from the run into the workspace
best_run.download_file('models/*.*')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model

# TODO: Change the `model_path`
model = Model.register(model_path="...",
                       model_name="...",
                       tags={'topic': "computer_vision", 'type': "classification"},
                       description="ANN model on MNIST dataset to classify handwritten digits.",
                       workspace=ws)

### Deploy an Azure Container Instance Web Service

In [ ]:
service_name = 'mnist-service'

# Use the `deployment_config` argument for custom config
service = Model.deploy(ws, service_name, [model], overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

### Test the Web Service Endpoint

In [ ]:
from utils import visualize
# TODO: Replace it with the consume script provided by AML studio
# Get input data (use the dumped json from the inference script) -> Show as img -> Inference
input_data = {
    "input_key": input_value
}

# Call the web service endpoint
result = service.run(input_data)
print("Web service response:", result)

# TODO: Get the predicted labels ID
predicted_labels = ...
# TODO: Get the test labels ID
test_labels = ...

visualize[result[0]]
print("Predicted labels: ", predicted_labels[0])
print("Ground truth: ", test_labels[0])

## Delete the Resource
TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
